### Imports

In [1]:
import os
import pandas
from flashtext import KeywordProcessor
import re

pandas.set_option('display.max_colwidth', None)

### 1 - Encontrar os DODFS que possuem os padrões dos atributos

#### Listagem dos arquivos e definição dos padrões em um DataFrame

In [10]:
dodfs_txt = []
for dirpath, subdirs, files in os.walk('../data/results/txt/'):
    for file in files:
        dodfs_txt.append(os.path.join(dirpath, file))

pattern_dict = {
    "Atributo": ["Hierarquia da Lotação", "Orgão", "Informação Errada", "Informação Corrigida", "Tipo de Ato Obrigatório"],
    "Padrão": ["Cargo em Comissao", "Hierarquia da Lotacao", "ONDE SE LE", "LEIA-SE", "ato que"]
}
patterns_table = pandas.DataFrame(data=pattern_dict)
patterns_table = patterns_table.style.set_properties(**{'text-align': 'left'})
patterns_table

,Atributo,Padrão
0,Hierarquia da Lotação,Cargo em Comissao
1,Orgão,Hierarquia da Lotacao
2,Informação Errada,ONDE SE LE
3,Informação Corrigida,LEIA-SE
4,Tipo de Ato Obrigatório,ato que


#### Filtrando os dodfs que possuem alguns dos padrões

In [3]:
files_found = []
keyword_processor = KeywordProcessor()
for index, row in patterns_table.iterrows():
    keyword_processor.add_keyword(row['Padrão'])
    
for file in dodfs_txt:
    text = open(file, 'r').read()
    keywords_found = keyword_processor.extract_keywords(text)
    if len(keywords_found) > 0:
        files_found.append((file, set(keywords_found)))

### 2 - Exploração do atributo Tipo de Ato: "ato que"

In [4]:
pos_verbs = set()
for file in files_found:
    if 'ato que' in file[1]:
        text = open(file[0], 'r').read()
        regex = r"ato que (\S+)"
        match = re.findall(regex, text)
        pos_verbs.update(match)

pos_verbs

{':',
 'AUTORIZOU',
 'AVERBOU',
 'Altera',
 'Autorizou',
 'Averbou',
 'CLARISSA',
 'CONCEDEU',
 'CONCENDEU',
 'CONVERTEU,',
 'Concedeu',
 'Constituiu',
 'DECLAROU',
 'DESAVERBOU',
 'DESIGNOU',
 'Declarou',
 'Designa',
 'Designou',
 'Disciplina',
 'Dispensou',
 'EVALDO',
 'EXONEROU,',
 'Exonerou',
 'Exonerou,',
 'ONDE',
 'PEDRO',
 'PUBLICOU',
 'RATIFICO',
 'RATIFICO,',
 'Suspendeu,',
 'TORNOU',
 'Tornou',
 'a',
 'alguns',
 'altere',
 'alterou',
 'antes',
 'anulou',
 'ao',
 'aplica',
 'aplicou',
 'aposentou',
 'apostilou,',
 'apresentar',
 'aprovou',
 'atente',
 'atingir',
 'atua',
 'autorizou',
 'autorizou,',
 'averbou',
 'cancelou',
 'causou',
 'chegou',
 'com',
 'cometer',
 'comparecer',
 'compareceu',
 'conceceu',
 'concede',
 'concedeu',
 'concedeu,',
 'concedia',
 'confere',
 'configure',
 'considerou',
 'consta',
 'constitua',
 'constitui',
 'constituiu',
 'contemple',
 'contraria',
 'converteu',
 'convoca',
 'convocou',
 'corresponde',
 'corrigiu',
 'cria',
 'criou',
 'culminou',

### 3 - Exploração dos Atributos de Informação errada e corrigida

Primeiramente checar se 'ONDE SE LE' sempre vem seguido de 'LEIA-SE'

In [5]:
not_both = 0
for file in files_found:
    if 'ONDE SE LE' in file[1] and 'LEIA-SE' not in file[1]:
        not_both += 1
        print(file[0])

../data/results/txt/2019/11_Novembro/DODF 217 13-11-2019 SUPLEMENTO.txt
../data/results/txt/2019/03_Março/DODF 044 07-03-2019 SUPLEMENTO.txt


O primeiro caso que a 'ONDE SE LE' não é seguido de 'LEIA-SE' é divido ao erro na escrita de 'LEIA SE'(sem hifen) e no segundo caso 'LEIA-SE' está separado por uma quebra de linha (LEIA-\nSE').

#### Extração de Informação errada e corrigida em pares

In [13]:
onde_se_le = []
leia_se = [] 
for file in files_found:
    if 'ONDE SE LE' in file[1] and 'LEIA-SE' in file[1]:
        text = open(file[0], 'r').read()
        regex = r'onde se le: "(.*?)", leia-se: "(.*?)"'
        match = re.findall(regex, text, re.IGNORECASE) 
        if len(match) > 0:
            for info in match:
                onde_se_le.append(info[0])
                leia_se.append(info[1])
                
info_correction = pandas.DataFrame({'ONDE SE LE': onde_se_le, 'LEIA-SE': leia_se})
info_correction  = info_correction .style.set_properties(**{'text-align': 'left'})
info_correction

,ONDE SE LE,LEIA-SE
0,"...Esta Portaria entra em vigor na data de sua publicacao, revogando as disposicoes em contrario, em especial, a Portaria no 293, de 18 de outubro de 2013 e suas alteracoes...",...Esta Portaria entra em vigor na data de sua publicacao...
1,...da Secretaria de Estado de Saude do Distrito Federal.,"...da Secretaria de Estado de Saude do Distrito Federal, a contar de 20 de dezembro de 2019."
2,"...da Secretaria de Estado de Saude do Distrito Federal, a contar de 16 de dezembro de 2019.","...da Secretaria de Estado de Saude do Distrito Federal, a contar de 11 de dezembro de 2019."
3,...matricula 1.682.701-35...,...1.682.701-5...
4,...matricula 165.885-25...,...matricula 103.912-1...
5,...COORDENACAO DE FISCALIZACAO TRIBUTARIA...,...COORDENACAO DE SISTEMAS TRIBUTARIOS...
6,... Processo: 00133- 00000146/2020-16...,...Processo: 00133- 00000149/2020-16...
7,...1.763 ... 3.963...,...1.757 ... 3.960...
8,"com as alteracoes introduzidas pela Lei no 7.475, de 13 de maio de 1986",na redacao do artigo 64 da Lei no 12.086/09
9,...5295 (cinco mil duzentos e noventa e cinco) dias...,...5103 (cinco mil cento e tres) dias...


In [11]:
retificar = []
onde_se_le = []
leia_se = [] 
for file in files_found:
    if 'ONDE SE LE' in file[1] and 'LEIA-SE' in file[1]:
        text = open(file[0], 'r').read()
        regex = r'RETIFICAR(.*?) onde se le: "(.*?)", leia-se: "(.*?)"'
        match = re.findall(regex, text, re.IGNORECASE) 
        if len(match) > 0:
            for info in match:
                retificar.append('RETIFICAR ' + info[0])
                onde_se_le.append(info[1])
                leia_se.append(info[2])
                
info_correction = pandas.DataFrame({'RETIFICAR': retificar, 'ONDE SE LE': onde_se_le, 'LEIA-SE': leia_se})
info_correction  = info_correction .style.set_properties(**{'text-align': 'left'})
info_correction

,RETIFICAR,ONDE SE LE,LEIA-SE
0,"RETIFICAR a Ordem de Servico no 04, de 09 de janeiro de 2020, publicada no DODF no 8, de 13 de janeiro de 2020, pagina18, o ato que averbou o tempo de servico prestado pelo servidor RICARDO PACHECO, matricula no 02406373, para incluir a seguinte redacao,",...5295 (cinco mil duzentos e noventa e cinco) dias...,...5103 (cinco mil cento e tres) dias...
1,"RETIFICAR a Portaria de 14 de marco de 2011, publicada no Diario Oficial do Distrito Federal no 51, de 16 de marco de 2011, pagina 53, o ato que averbou o tempo de contribuicao de ELOISA SALVADORA LOPES DE SOUZA, matricula no 42.827-2,","...no total de 5.045 (cinco mil e cento e quarenta e cinco) dias, referentes aos periodos de 01/11/80 a 28/02/81; 01/07/81 a 18/03/82; 19/03/82 a 25/05/88; 01/06/88 a 31/10/92, e, de 01/12/92 a 31/08/94...","...no total de 2.630 (dois mil, seiscentos e trinta) dias, referentes aos periodos de 01/11/80 a 28/02/81; 01/07/81 a 18/03/82; 01/06/88 a 31/10/92; 01/12/92 a 31/08/94..."
2,"RETIFICAR , na Portaria no 168, de 21 de fevereiro de 2020, publicada no DODF no 38 de 27 de fevereiro de 2020, pagina 30, o ato que designou ANA PAULA SOUSA MAFRA, matricula no 171.952-1,como substituta automatica do cargo de Gerente, da Gerencia de Seguranca, da Unidade de Internacao de Planaltina, da Coordenacao de Internacao, da Subsecretaria do Sistema Socioeducativo, da Secretaria de Estado de Justica e Cidadania.",...da Gerencia de Seguranca...,...da Gerencia de Saude...
3,"RETIFICAR , na Portaria no 250, de 27 de marco de 2020, publicada no DODF no 60 de 30 de marco de 2020, pagina 51, o ato que suspende, por necessidade do servico, a contar de 23 de marco de 2020,","...SUSPENDE, por necessidade do servico, a contar de 23 de marco de 2020, as ferias do servidor MARCOS VITAL MOREIRA DORNELAS, Agente Socioeducativo, matricula 196.077-6, ocupante do Cargo de Natureza Especial, Simbolo CNE-07, de Vice Diretor da Unidade de Internacao de Sao Sebastiao, da Subsecretaria do Sistema Socioeducativo, da Secretaria de Estado de Justica e Cidadania do Distrito Federal, marcadas para o periodo de 18 a 27 de marco de 2020, ficando assegurado o seu gozo posterior...","...SUSPENDER o gozo de ferias da servidor JALLES GONCALVES DOS REIS, matricula no 1.431.283-2, Diretor, da Diretoria de Contratacoes, da Secretaria do Estado de Justica e Cidadania, marcadas para o periodo de 27/03/2020 a 05/04/2020, a contar de 28/03/2020, por necessidade de servico e em virtude da atual situacao de emergencia em saude publica e pandemia declarada pela Organizacao Mundial de Saude (OMS), em decorrencia do coronavirus (COVID-19), ficando assegurado o gozo posterior..."
4,"RETIFICAR , na Ordem de Servico de 8 de abril 2003, publicada no DODF no 73, de 15 de abril de 2003, pag. 18, e na Ordem de Servico no 3 de 2 de marco de 2009, publicada no DODF no 49, de 12 de marco de 2009, pag. 36/37, que concedeu Licenca Premio por Assiduidade ao servidor MANUEL ALVES PIRES, matricula 091.200-X,","...1o quinquenio, referente ao periodo de 06/02/98 a 05/02/2003, 2o quinquenio, MANOEL ALVES PIRES...","...1o quinquenio, referente ao periodo de 06/02/1998 a 04/02/2003, 2o quinquenio, MANUEL ALVES PIRES..."
5,"RETIFICAR na Portaria no 247, de 1o de julho de 2019, publicada no DODF no 123, de 3 de julho de 2019, pagina 32, o ato que se refere a concessao de promocao funcional nos termos do Decreto no 37.770, de 14 de novembro de 2016 e do artigo 8o da Lei no 5.193, de 26 de setembro de 2013, da servidora HELI APARECIDA DE BARROS, 172.486-X, Analista de Politicas Publicas e Gestao Governamental,","...2a V, 45,00; 40,00; 85,00, 1a I, 13/12/2018...","...2a V, 45,00; 40,00; 85,00, 1a I, 23/11/2018 ..."
6,"RETIFICAR a Portaria de 12 de janeiro de 2005, publicado no DODF no 13 de 19 de janeiro de 2005, pagina 21, que concedeu Licenca Premio por Assiduidade, a servidora PATRICIA ARAUJO DA SILVA, matricula no 43.065-X,","...2o quinquenio, no periodo de 03 de outubro de 1999 a 01 outubro d

Nesses resultados falta sobre qual conteudo as informações serão corrigidas

#### Extração dos Orgão e das Herarquias de Lotação

In [12]:
cargos = []
hierarquias = []
orgaos = []

for file in files_found[:10]:
    if 'Cargo em Comissao' in file[1]:
        text = open(file[0], 'r').read()
        regex = r'[c|C]argo em Comissao, Simbolo (.*?), d[a|e|o] (.*?), d[a|e|o] ([^.]*,?), d[a|e|o] (.*?)\.'
        match = re.findall(regex, text) 
        if len(match) > 0:
            for info in match:
                cargos.append(info[1])
                hierarquias.append(info[2])
                orgaos.append(info[3])
                
cargos_em_comissao = pandas.DataFrame({'Cargo': cargos, 'Hierarquia': hierarquias, 'Orgaos': orgaos})
cargos_em_comissao = cargos_em_comissao.style.set_properties(**{'text-align': 'left'})
cargos_em_comissao

,Cargo,Hierarquia,Orgaos
0,Gerente,"Gerencia de Acesso e Qualidade em Atencao Primaria a Saude, da Diretoria Regional de Atencao Primaria a Saude, da Superintendencia da Regiao de Saude Leste",Secretaria de Estado de Saude do Distrito Federal
1,Supervisor de Servicos de Atencao Primaria,"Gerencia de Servicos de Atencao Primaria no 4 de Sao Sebastiao, da Diretoria Regional de Atencao Primaria a Saude, da Superintendencia da Regiao de Saude Leste",Secretaria de Estado de Saude do Distrito Federal
2,Assessor Tecnico,Coordenacao Regional de Ensino de Brazlandia,Secretaria de Estado de Educacao do Distrito Federal
3,Assessor Tecnico,"Nucleo de Engenharia, da Gerencia de Servicos Gerais, da Diretoria de Apoio Operacional, da Subsecretaria de Administracao Geral",Defensoria Publica do Distrito Federal
4,Assessor,Assessoria de Relacoes Institucionais,Secretaria de Estado de Educacao do Distrito Federal
5,Assessor Tecnico,Subsecretaria de Infraestrutura e Apoio Educacional,Secretaria de Estado de Educacao do Distrito Federal
6,Chefe,"Nucleo de Parasitologia e Micologia, da Gerencia de Biologia Medica, da Diretoria do Laboratorio Central de Saude Publica, da Subsecretaria de Vigilancia a Saude",Secretaria de Estado de Saude do Distrito Federal
7,Supervisor de Unidade,"Gerencia de Assistencia Clinica, da Diretoria do Hospital Regional de Taguatinga, da Superintendencia da Regiao de Saude Sudoeste",Secretaria de Estado de Saude do Distrito Federal
8,Supervisor de Unidade,"Gerencia de Assistencia Clinica, da Diretoria do Hospital Regional de Taguatinga, da Superintendencia da Regiao de Saude Sudoeste",Secretaria de Estado de Saude do Distrito Federal
9,Chefe,"Nucleo de Nutricao e Dietetica, da Gerencia de Assistencia Multidisciplinar e Apoio Diagnostico, da Diretoria do Hospital da Regiao Leste, da Superintendencia da Regiao de Saude Leste",Secretaria de Estado de Saude do Distrito Federal
